In [1]:
import pandas as pd

In [19]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD, Adam
from keras import backend as K
K.set_image_data_format('channels_first')

In [20]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [21]:
from glob import glob
import numpy as np
from keras.preprocessing import image

In [22]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

In [23]:
from skimage import color, exposure, transform

In [33]:
#Parameters to tune
IMG_SIZE = 350
batch_size = 32
epochs = 30
NUM_CLASSES = 3
NUM_IMAGES = 1000

In [34]:
def get_batches(path, gen=image.ImageDataGenerator(width_shift_range=0.3,height_shift_range=0.3), shuffle=True, batch_size=8, class_mode='categorical'):
    """
        Takes the path to a directory, and generates batches of augmented/normalized data. Yields batches indefinitely, in an infinite loop.

        See Keras documentation: https://keras.io/preprocessing/image/
    """
    return gen.flow_from_directory(path, target_size=(IMG_SIZE,IMG_SIZE),
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [41]:
def cnn_model():
    model = Sequential()
    #Block 1
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(3, IMG_SIZE, IMG_SIZE), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    #Block2
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    #Block 3
    model.add(Conv2D(128, (3, 3), padding='same',activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    #Block 4
#     model.add(Conv2D(128, (3, 3), padding='same',activation='relu'))
#     model.add(Conv2D(128, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.2))
    
    #FC Block
    model.add(Flatten())
#     model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

In [42]:
def fit(model, batches, val_batches, nb_epoch=epochs):
        """
            Fits the model on data yielded batch-by-batch by a Python generator.
            See Keras documentation: https://keras.io/models/model/
        """
        return model.fit_generator(batches, steps_per_epoch=NUM_IMAGES/epochs,
                validation_data=val_batches, validation_steps = 550/30, epochs=nb_epoch)

In [43]:
model=cnn_model()

In [44]:
# let's train the model using SGD + momentum
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [45]:
path =''
batches = get_batches('train', batch_size=batch_size)
val_batches = get_batches(path+'val', batch_size=batch_size)

Found 2228 images belonging to 3 classes.
Found 577 images belonging to 3 classes.


In [46]:
fit_model = fit(model, batches, val_batches, nb_epoch=1)

Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[3,3,64,64]
	 [[Node: training_3/Adam/mul_33 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](Adam_3/beta_2/read, training_3/Adam/Variable_22/read)]]
	 [[Node: metrics_3/acc/Mean/_131 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1816_metrics_3/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'training_3/Adam/mul_33', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.